In [144]:
import pandas as pd
import numpy as np

In [251]:
slots = pd.read_csv('DataFiles/NCAATourneySlots.csv')
seeds = pd.read_csv('DataFiles/NCAATourneySeeds.csv',index_col=['Season','Seed'])

In [252]:
tourney = pd.read_csv('DataFiles/NCAATourneyCompactResults.csv')
tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [253]:
predict = pd.read_csv('submission_2019-3-10_20:47.csv')
predict['Season'] = predict['ID'].str.split('_').apply(lambda x: x[0]).astype(int)
predict['TeamA'] = predict['ID'].str.split('_').apply(lambda x: x[1]).astype(int)
predict['TeamB'] = predict['ID'].str.split('_').apply(lambda x: x[2]).astype(int)
predict = predict.set_index(['Season','TeamA','TeamB'])

In [320]:
#######################
results = slots.copy()
results = results[results['Season'] == 2018]
results['Round'] = results['Slot'].str.slice(stop=2)
results['TrueSS'] = np.nan
results['TrueWS'] = np.nan
for r in ['W1','X1','Y1','Z1']+['R%d' % (n+1) for n in range(6)]:
    for i,game in results[results['Round'] == r].iterrows():
        teamSS = game['TrueSS']
        teamWS = game['TrueWS']
        if np.isnan(teamSS):
            teamSS = seeds.loc[game['Season'],game['StrongSeed']]['TeamID']
            results.at[i,'TrueSS'] = teamSS
        if np.isnan(teamWS):
            teamWS = seeds.loc[game['Season'],game['WeakSeed']]['TeamID']
            results.at[i,'TrueWS'] = teamWS
        
        match = tourney[(((tourney['WTeamID'] == teamSS) & (tourney['LTeamID'] == teamWS)) | \
                        ((tourney['WTeamID'] == teamWS) & (tourney['LTeamID'] == teamSS))) & \
                        (tourney['Season'] == game['Season'])]
        
        if match['WTeamID'].values[0] == teamSS:
            results.at[i,'Result'] = 1 
            results.at[i,'ScoreSS'] = match['WScore'].values[0]
            results.at[i,'ScoreWS'] = match['LScore'].values[0]
        else:
            results.at[i,'Result'] = 0
            results.at[i,'ScoreSS'] = match['LScore'].values[0]
            results.at[i,'ScoreWS'] = match['WScore'].values[0]
        
        
        if r != 'R6':
            idx = results[((results['StrongSeed'] == game['Slot']) | (results['WeakSeed'] == game['Slot'])) & \
            (results['Season'] == 2018)].index[0]
            
            if results.loc[idx,'StrongSeed'] == game['Slot']:
                #print(results.loc[idx,'StrongSeed'],game['Slot'])
                results.at[idx,'TrueSS'] = match['WTeamID']
            else:
                #print(results.loc[idx,'WeakSeed'],game['Slot'])
                results.at[idx,'TrueWS'] = match['WTeamID']
        #print(i,idx)
        #print()